In [ ]:
# SMS Scam Detection - Llama Model Fine-tuning
# =============================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import time
import re
import os
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    LlamaTokenizer,
    LlamaForSequenceClassification,
    BitsAndBytesConfig
)

from peft import (
    LoraConfig,
    TaskType,
    get_peft_model,
    prepare_model_for_kbit_training
)

from datasets import Dataset, DatasetDict
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    matthews_corrcoef, roc_auc_score, average_precision_score,
    confusion_matrix, classification_report, roc_curve, precision_recall_curve
)
import evaluate
import warnings
from pathlib import Path
from tqdm.auto import tqdm

# Set plotting style
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 8)

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set CUDA_LAUNCH_BLOCKING for better error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Ignore specific warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers")

# Setup Hugging Face token for model access
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

# Set up project paths
project_dir = '/content/drive/MyDrive/sms-scam-detection'
os.chdir(project_dir)

data_dir = "data/processed/"
model_dir = "models/llm_os/"  # Open Source LLMs
results_dir = "results/"

# Create directories
os.makedirs(model_dir, exist_ok=True)
os.makedirs(os.path.join(results_dir, "metrics"), exist_ok=True)
os.makedirs(os.path.join(results_dir, "visualizations"), exist_ok=True)

# Setup label mapping
id2label = {0: "Legitimate", 1: "Scam"}
label2id = {"Legitimate": 0, "Scam": 1}

def load_data():
    """Load and prepare the dataset."""
    train_df = pd.read_csv(os.path.join(data_dir, "train.csv"))
    val_df = pd.read_csv(os.path.join(data_dir, "val.csv"))
    test_df = pd.read_csv(os.path.join(data_dir, "test.csv"))

    print(f"Loaded data: Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")

    # Print class distribution
    print("\nClass Distribution:")
    for name, df in [("Training", train_df), ("Validation", val_df), ("Test", test_df)]:
        print(f"{name} Set:")
        print(df['label'].value_counts(normalize=True) * 100)

    # Calculate imbalance ratio
    train_neg_count = (train_df['label'] == 0).sum()
    train_pos_count = (train_df['label'] == 1).sum()
    imbalance_ratio = train_neg_count / train_pos_count if train_pos_count > 0 else float('inf')
    print(f"\nImbalance ratio (legitimate:scam): {imbalance_ratio:.2f}:1")

    # Convert to Hugging Face datasets
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)
    test_dataset = Dataset.from_pandas(test_df)

    dataset_dict = DatasetDict({
        "train": train_dataset,
        "valid": val_dataset,
        "test": test_dataset
    })

    print("\nDataset structure:")
    print(dataset_dict)

    return dataset_dict

def preprocess_function(examples, tokenizer, max_length=128):
    """Tokenize and preprocess text examples."""
    text_column = "cleaned_text" if "cleaned_text" in examples else "message"
    return tokenizer(
        examples[text_column],
        max_length=max_length,
        padding='max_length',
        truncation=True
    )

def tokenize_data(dataset_dict, tokenizer, max_length=128):
    """Tokenize all splits in a dataset dictionary."""
    return dataset_dict.map(
        lambda examples: preprocess_function(examples, tokenizer, max_length),
        batched=True
    )

def compute_metrics(eval_pred):
    """Compute evaluation metrics for model predictions."""
    predictions, labels = eval_pred

    probabilities = torch.nn.functional.softmax(torch.tensor(predictions), dim=-1).numpy()
    positive_class_probs = probabilities[:, 1]
    predicted_classes = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(labels, predicted_classes)
    precision = precision_score(labels, predicted_classes)
    recall = recall_score(labels, predicted_classes)
    f1 = f1_score(labels, predicted_classes)
    mcc = matthews_corrcoef(labels, predicted_classes)
    roc_auc = roc_auc_score(labels, positive_class_probs)
    pr_auc = average_precision_score(labels, positive_class_probs)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "mcc": mcc,
        "roc_auc": roc_auc,
        "pr_auc": pr_auc
    }

def create_data_collator(tokenizer):
    """Create a data collator for batching examples."""
    return DataCollatorWithPadding(tokenizer=tokenizer)

def plot_confusion_matrix(cm, classes, model_name):
    """Plot confusion matrix."""
    results_dir = Path(f"results/{model_name}")
    results_dir.mkdir(parents=True, exist_ok=True)

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title(f"{model_name} - Confusion Matrix")
    plt.tight_layout()
    plt.savefig(results_dir / "confusion_matrix.png")
    plt.show()

def plot_roc_curve(fpr, tpr, roc_auc, model_name):
    """Plot ROC curve."""
    results_dir = Path(f"results/{model_name}")
    results_dir.mkdir(parents=True, exist_ok=True)

    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC Curve (AUC = {roc_auc:.3f})')
    plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"{model_name} - ROC Curve")
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(results_dir / "roc_curve.png")
    plt.show()

def plot_pr_curve(precision, recall, pr_auc, model_name):
    """Plot precision-recall curve."""
    results_dir = Path(f"results/{model_name}")
    results_dir.mkdir(parents=True, exist_ok=True)

    plt.figure(figsize=(10, 8))
    plt.step(recall, precision, color='green', lw=2, where='post',
             label=f'PR Curve (AUC = {pr_auc:.3f})')
    plt.fill_between(recall, precision, alpha=0.2, color='green', step='post')
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title(f"{model_name} - Precision-Recall Curve")
    plt.legend(loc="lower left")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(results_dir / "pr_curve.png")
    plt.show()

def setup_quantization_config():
    """Setup 4-bit quantization configuration for memory efficiency."""
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )
    return quantization_config

def setup_lora_config(model, lora_r=16, lora_alpha=32, lora_dropout=0.1):
    """Setup LoRA configuration for parameter-efficient fine-tuning."""
    # Determine target modules based on model architecture
    if "llama" in model.config._name_or_path.lower():
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]
    elif "mistral" in model.config._name_or_path.lower():
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]
    elif "falcon" in model.config._name_or_path.lower():
        target_modules = ["query_key_value"]
    elif "phi" in model.config._name_or_path.lower():
        target_modules = ["Wqkv", "out_proj"]
    else:
        # Default for most decoder architectures
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]

    # Define LoRA config
    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=lora_r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        target_modules=target_modules,
        bias="none"
    )

    # Apply LoRA
    lora_model = get_peft_model(model, peft_config)

    # Print trainable parameters comparison
    print("Trainable parameters:")
    trainable_params = sum(p.numel() for p in lora_model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in lora_model.parameters())
    print(f"  - LoRA trainable params: {trainable_params:,} ({trainable_params/total_params:.2%} of total)")

    return lora_model

def train_model(model, tokenizer, tokenized_data, training_args, model_name):
    """Train a model and evaluate it on validation data."""
    print(f"\n=== Training {model_name} ===")

    # Create results directory
    results_dir = Path(f"results/{model_name}")
    results_dir.mkdir(parents=True, exist_ok=True)

    # Set up trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_data["train"],
        eval_dataset=tokenized_data["valid"],
        tokenizer=tokenizer,
        data_collator=create_data_collator(tokenizer),
        compute_metrics=compute_metrics,
    )

    # Train the model
    start_time = time.time()
    train_result = trainer.train()
    end_time = time.time()
    training_time = end_time - start_time
    training_time_minutes = round(training_time / 60, 2)

    print(f"\nTraining completed in {training_time_minutes} minutes")
    print(f"Training Loss: {train_result.training_loss:.4f}")

    # Evaluate on validation set
    eval_results = trainer.evaluate()
    print("\nValidation Results:")
    for key, value in eval_results.items():
        print(f"{key}: {value:.4f}")

    # Save the model
    model_save_path = results_dir / "final_model"
    trainer.save_model(str(model_save_path))
    print(f"Model saved to {model_save_path}")

    # Save training metrics
    metrics_file = results_dir / "training_metrics.json"
    with open(metrics_file, "w") as f:
        json.dump({
            "training_time_seconds": training_time,
            "training_time_minutes": training_time_minutes,
            "final_train_loss": train_result.training_loss,
            "eval_results": eval_results
        }, f, indent=4)

    return trainer

def evaluate_model(trainer, tokenized_test_data, model_name):
    """Evaluate a trained model on test data."""
    print(f"\n=== Evaluating {model_name} on Test Set ===")

    # Make predictions on test set
    test_results = trainer.predict(tokenized_test_data)

    predictions = test_results.predictions
    labels = test_results.label_ids

    # Calculate probabilities and predicted classes
    probabilities = torch.nn.functional.softmax(torch.tensor(predictions), dim=-1).numpy()
    positive_class_probs = probabilities[:, 1]
    predicted_classes = np.argmax(predictions, axis=1)

    # Calculate metrics
    accuracy = accuracy_score(labels, predicted_classes)
    precision = precision_score(labels, predicted_classes)
    recall = recall_score(labels, predicted_classes)
    f1 = f1_score(labels, predicted_classes)
    mcc = matthews_corrcoef(labels, predicted_classes)
    roc_auc = roc_auc_score(labels, positive_class_probs)
    pr_auc = average_precision_score(labels, positive_class_probs)
    cm = confusion_matrix(labels, predicted_classes)

    # Print results
    print("\nTest Metrics:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Matthews Correlation Coefficient: {mcc:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")
    print(f"PR AUC: {pr_auc:.4f}")

    # Print detailed classification report
    print("\nClassification Report:")
    class_names = list(id2label.values())
    report = classification_report(labels, predicted_classes, target_names=class_names)
    print(report)

    # Save results
    results_dir = Path(f"results/{model_name}")
    results_dir.mkdir(parents=True, exist_ok=True)

    # Save metrics as CSV
    metrics_df = pd.DataFrame({
        'Model': [model_name],
        'Accuracy': [accuracy],
        'Precision': [precision],
        'Recall': [recall],
        'F1': [f1],
        'MCC': [mcc],
        'ROC_AUC': [roc_auc],
        'PR_AUC': [pr_auc]
    })
    metrics_df.to_csv(results_dir / "test_metrics.csv", index=False)

    # Save classification report
    with open(results_dir / "classification_report.txt", "w") as f:
        f.write(report)

    # Create visualizations
    plot_confusion_matrix(cm, class_names, model_name)

    # ROC curve
    fpr, tpr, _ = roc_curve(labels, positive_class_probs)
    plot_roc_curve(fpr, tpr, roc_auc, model_name)

    # Precision-Recall curve
    precision_points, recall_points, _ = precision_recall_curve(labels, positive_class_probs)
    plot_pr_curve(precision_points, recall_points, pr_auc, model_name)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "mcc": mcc,
        "roc_auc": roc_auc,
        "pr_auc": pr_auc,
        "confusion_matrix": cm,
        "predicted_classes": predicted_classes,
        "probabilities": probabilities
    }

def predict_examples(model, tokenizer, examples, model_name):
    """Predict on a list of example messages."""
    model.eval()
    results = []

    # Process each example
    for text in examples:
        # Tokenize the text
        inputs = tokenizer(
            text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128
        ).to(device)

        # Make prediction
        with torch.no_grad():
            outputs = model(**inputs)

        # Calculate probabilities and prediction
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(probs, dim=1).item()
        prob_score = probs[0][predicted_class].item()

        results.append({
            "text": text,
            "predicted_class": id2label[predicted_class],
            "confidence": prob_score
        })

    # Convert to DataFrame for easy viewing
    results_df = pd.DataFrame(results)

    print(f"\n=== Example Predictions with {model_name} ===")
    for i, row in results_df.iterrows():
        print(f"\nExample {i+1}:")
        print(f"Text: {row['text']}")
        print(f"Prediction: {row['predicted_class']}")
        print(f"Confidence: {row['confidence']:.4f}")

    return results_df

def main():
    """Main execution function."""
    # Load data
    dataset_dict = load_data()

    # Available Llama-based models (adjust based on availability and resources)
    model_configs = [
        {
            "name": "llama2_7b_lora",
            "model_path": "meta-llama/Llama-2-7b-hf",
            "use_quantization": True,
            "use_lora": True
        }
    ]

    all_results = {}

    # Process each model configuration
    for config in model_configs:
        model_name = config["name"]
        model_path = config["model_path"]
        use_quantization = config.get("use_quantization", False)
        use_lora = config.get("use_lora", False)

        print(f"\n{'='*60}")
        print(f"Processing {model_name}")
        print(f"Model: {model_path}")
        print(f"Quantization: {use_quantization}")
        print(f"LoRA: {use_lora}")
        print(f"{'='*60}")

        try:
            # Setup quantization config if needed
            quantization_config = setup_quantization_config() if use_quantization else None

            # Load tokenizer
            print("Loading tokenizer...")
            tokenizer = AutoTokenizer.from_pretrained(
                model_path,
                use_auth_token=hf_token,
                trust_remote_code=True
            )

            # Add padding token if not present
            if tokenizer.pad_token is None:
                tokenizer.pad_token = tokenizer.eos_token

            # Load model
            print("Loading model...")
            model = AutoModelForSequenceClassification.from_pretrained(
                model_path,
                num_labels=2,
                id2label=id2label,
                label2id=label2id,
                quantization_config=quantization_config,
                device_map="auto" if use_quantization else None,
                torch_dtype=torch.float16 if use_quantization else torch.float32,
                use_auth_token=hf_token,
                trust_remote_code=True
            )

            # Prepare model for training if using quantization
            if use_quantization:
                model = prepare_model_for_kbit_training(model)

            # Apply LoRA if specified
            if use_lora:
                print("Applying LoRA configuration...")
                model = setup_lora_config(model)

            # Move to device if not using quantization
            if not use_quantization:
                model = model.to(device)

            # Tokenize data
            print("Tokenizing data...")
            tokenized_data = tokenize_data(dataset_dict, tokenizer, max_length=128)

            # Define training arguments
            training_args = TrainingArguments(
                output_dir=f"results/{model_name}/checkpoints",
                num_train_epochs=3,
                per_device_train_batch_size=4,  # Small batch size for memory efficiency
                per_device_eval_batch_size=4,
                gradient_accumulation_steps=4,  # Effective batch size = 4*4 = 16
                warmup_steps=100,
                weight_decay=0.01,
                logging_dir=f"results/{model_name}/logs",
                logging_steps=50,
                eval_strategy="epoch",
                save_strategy="epoch",
                save_total_limit=1,
                load_best_model_at_end=True,
                metric_for_best_model="mcc",
                greater_is_better=True,
                push_to_hub=False,
                dataloader_pin_memory=False,
                remove_unused_columns=False,
                fp16=True,  # Use mixed precision for memory efficiency
                gradient_checkpointing=True,  # Trade compute for memory
            )

            # Train the model
            trainer = train_model(model, tokenizer, tokenized_data, training_args, model_name)

            # Evaluate on test set
            test_results = evaluate_model(trainer, tokenized_data["test"], model_name)

            # Store results
            all_results[model_name] = test_results

            # Test on example messages
            example_messages = [
                "Congratulations! You've won ugx100000. Click here to claim: www.example.com",
                "Your account has been suspended. Please verify your identity by sending your PIN.",
                "Hi, just checking if we're still meeting for lunch tomorrow at 12?",
                "URGENT: Your payment of ugx55000 has been processed. If this was not you, call immediately.",
                "Your package will be delivered tomorrow between 10am and 2pm. No signature required.",
                "You have been selected to win ugx500000! Call now to claim your prize before it expires!",
                "Meeting postponed to next Wednesday at 3pm. Please confirm if you can attend."
            ]

            # Make predictions on examples
            example_predictions = predict_examples(model, tokenizer, example_messages, model_name)

            # Save example predictions
            example_predictions.to_csv(f"results/{model_name}/example_predictions.csv", index=False)

            # Clean up GPU memory
            del model
            del trainer
            torch.cuda.empty_cache()

        except Exception as e:
            print(f"Error processing {model_name}: {str(e)}")
            continue

    # Compile and save results
    if all_results:
        # Create comparison DataFrame
        comparison_results = []
        for model_name, results in all_results.items():
            comparison_results.append({
                "Model": model_name,
                "Model_Type": "LLM_OS",  # Open Source LLM
                "Accuracy": results["accuracy"],
                "Precision": results["precision"],
                "Recall": results["recall"],
                "F1": results["f1"],
                "MCC": results["mcc"],
                "ROC_AUC": results["roc_auc"],
                "PR_AUC": results["pr_auc"]
            })

        comparison_df = pd.DataFrame(comparison_results)
        comparison_df = comparison_df.sort_values("MCC", ascending=False)

        # Save Llama results
        comparison_df.to_csv("results/metrics/llama_results.csv", index=False)

        print("\n" + "="*60)
        print("LLAMA MODEL RESULTS")
        print("="*60)
        print(comparison_df.to_string(index=False))

        # Load and combine with previous results if available
        all_models_path = "results/metrics/final_all_models_comparison.csv"

        try:
            existing_df = pd.read_csv(all_models_path)

            # Combine results
            combined_df = pd.concat([existing_df, comparison_df], ignore_index=True)
            combined_df = combined_df.sort_values("MCC", ascending=False)

            # Save updated results
            combined_df.to_csv(all_models_path, index=False)

            print("\n" + "="*80)
            print("UPDATED FINAL COMPARISON OF ALL MODELS")
            print("="*80)
            key_columns = ["Model", "Model_Type", "MCC", "F1", "ROC_AUC", "PR_AUC"]
            available_columns = [col for col in key_columns if col in combined_df.columns]
            print(combined_df[available_columns].head(10).to_string(index=False))

            # Create updated visualization
            plt.figure(figsize=(16, 10))

            # Focus on key metrics
            key_metrics = ["F1", "MCC", "ROC_AUC", "PR_AUC"]
            available_metrics = [col for col in key_metrics if col in combined_df.columns]

            melted_df = combined_df.melt(
                id_vars=["Model"],
                value_vars=available_metrics,
                var_name="Metric",
                value_name="Score"
            )

            sns.barplot(data=melted_df, x="Model", y="Score", hue="Metric")
            plt.title("Final Performance Comparison: All Models Including Llama")
            plt.xticks(rotation=45, ha='right')
            plt.ylim(0, 1)
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.savefig("results/visualizations/final_all_models_with_llama.png", dpi=300, bbox_inches='tight')
            plt.show()

            # Print top 5 models
            print(f"\n🏆 TOP 5 MODELS BY MCC:")
            top_5 = combined_df.head(5)
            for i, (_, row) in enumerate(top_5.iterrows(), 1):
                model_type = row.get('Model_Type', 'Unknown')
                print(f"{i}. {row['Model']} ({model_type}): MCC = {row['MCC']:.4f}, F1 = {row['F1']:.4f}")

        except FileNotFoundError:
            print("No previous model results found for comparison.")

            # Create new comparison file with just this model
            new_df = pd.DataFrame({
                'Model': [model_name],
                'Model_Type': ['LLM_OS'],  # Open Source LLM
                'Accuracy': [results['accuracy']],
                'Precision': [results['precision']],
                'Recall': [results['recall']],
                'F1': [results['f1']],
                'MCC': [results['mcc']],
                'ROC_AUC': [results['roc_auc']],
                'PR_AUC': [results['pr_auc']]
            })

            # Save results
            new_df.to_csv(all_models_path, index=False)

            print("\nModel performance:")
            print(new_df[['Model', 'Model_Type', 'MCC', 'F1', 'Accuracy']])

    print(f"\nLlama model fine-tuning completed successfully!")
    print(f"Results saved to: results/")

if __name__ == "__main__":
    main()